# Tarea 2: análisis de sentimientos usando aprendizaje supervisado

##### Por: Daniela Flores Villanueva 

## Sobre las librerías

En primer lugar, se cargan todas las librerías empleadas en esta tarea:
- `pandas`: librería utilizada para cargar la matriz GloVe y posteriormente, para confeccionar un `DataFrame` con palabras y su sentimiento asociado.
- `csv`: utilizada para cargar los vectores, según lo dispuesto en [este](https://stackoverflow.com/questions/37793118/load-pretrained-glove-vectors-in-python) sitio.
- `nltk`: se usó para cargar el *opinion lexicon* descrito en el enunciado de manera conveniente. Se probó además sus métodos para realizar lematización, proceso que consiste en, dada una palabra flexionada (en forma plural, verbo conjugado, etc), encontrar su lema, es decir, algo similar a la forma en que sería encontrada en un diccionario.
- `string`: para obtener signos de puntuación, con el objeto de pre-procesar oraciones.
- `sklearn`: se utilizaron sus implementaciones de *Support Vector Machines*, *K-Nearest Neighbor* y *Random Forest* para construir clasificadores para el sentimiento de palabras y posteriormente oraciones. Se aprovechó también `train_test_split` para separar los datos en conjuntos de entrenamiento y de prueba, `GridSearchCV` para la calibración de hiperparámetros y los métodos `accuracy_score`, `precision_score`, `recall_score` y `f1-score` para cuantificar el desempeño de los modelos. Finalmente `joblib` se utiliza para serializar y guardar los modelos entrenados.

In [1]:
import pandas as pd
import csv
from nltk.stem import WordNetLemmatizer
import numpy as np
import nltk
nltk.download("stopwords")
nltk.download("opinion_lexicon")
nltk.download('punkt')
from nltk.corpus import opinion_lexicon
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.externals import joblib

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/DanielaFlores/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package opinion_lexicon to
[nltk_data]     /Users/DanielaFlores/nltk_data...
[nltk_data]   Package opinion_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/DanielaFlores/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

In [3]:
GLOVE_PATH = "./glove.42B.300d.txt"

In [4]:
NON_WORDS = list(punctuation)
NON_WORDS.extend(map(str, range(10)))

In [5]:
ENGLISH_STOPWORDS = stopwords.words("english")

La forma de cargar el opinion lexicon es la descrita en la [documentación](http://www.nltk.org/api/nltk.corpus.reader.html?highlight=wordnet) de NLTK.

In [6]:
positive_data = opinion_lexicon.positive()
negative_data = opinion_lexicon.negative()

Con el fin de tener una representación más general de las palabras, se convierte cada vocablo en las listas anteriores a minúsculas.

In [7]:
positive_data = [word.lower() for word in positive_data]
negative_data = [word.lower() for word in negative_data]

A continuación, se carga la matriz de *embeddings*:

In [8]:
glove_matrix = pd.read_csv(GLOVE_PATH, sep=" ", index_col=0, header=None, quoting=csv.QUOTE_NONE, na_values=None, keep_default_na=False)

En el mismo recurso para cargar la matriz en memoria (adjunto en la descripción de las librerías utilizadas), se menciona una función que, dada una palabra, retorna su *embedding*. Esta función es `vec(w)`, definida a continuación:

In [9]:
def vec(w): 
    return glove_matrix.loc[w].as_matrix()

## Breve análisis exploratorio

No se puede empezar a trabajar sin tener al menos una noción básica de la composición de la matriz y de las palabras con etiqueta. Por esta razón, en primer lugar cabe preguntarse cuántas palabras hay en la matriz de *embeddings*.

In [10]:
glove_words = set(glove_matrix.index.tolist())
print(len(glove_words))

1917494


Podría ocurrir que no todas las palabras que tenemos etiquetadas según su polaridad tengan su representación vectorial en GloVe, por lo que conviene realizar el siguiente análisis:
Dada la unión entre las palabras de polaridad positiva y las de polaridad negativa, se revisa si están o no en la matriz. De no estar, se agregan a la lista `missing_words`, para decidir qué hacer con ellas futuramente.

In [11]:
missing_words = set()
for word in set(positive_data).union(set(negative_data)):
    if word not in glove_words:
        missing_words.add(word)
print(len(missing_words))

161


Es posible notar que hay 161 palabras del *opinion lexicon* para las que no se tiene una representación vectorial. Antes de decidir qué hacer con estas palabras faltantes, conviene preguntarse si existen palabras que estén etiquetadas tanto de polaridad positiva como de polaridad negativa.

In [12]:
wrong_tag = set(positive_data).intersection(negative_data)
print(wrong_tag)

{'envious', 'enviously', 'enviousness'}


Así, se evidencia que tres palabras relacionadas a la envidia parecen estar mal etiquetadas, pues no deberían aparecer entre las palabras positivas.

A continuación, se procede a realizar lematización de los vocablos positivos y negativos, para ver si existe algún cambio en la cantidad palabras sin *embedding* tras este cambio.

In [13]:
positive_data_lemmatized = {nltk.stem.WordNetLemmatizer().lemmatize(word) for word in positive_data}
negative_data_lemmatized = {nltk.stem.WordNetLemmatizer().lemmatize(word) for word in negative_data}

In [14]:
missing_words_lemmatized = set()
for word in positive_data_lemmatized.union(set(negative_data_lemmatized)):
    if word not in glove_words:
        missing_words_lemmatized.add(word)
print(len(missing_words_lemmatized))

160


In [15]:
wrong_tag_lemmatized = set(positive_data_lemmatized).intersection(negative_data_lemmatized)
print(wrong_tag_lemmatized)

{'envious', 'enviously', 'plea', 'enviousness'}


In [16]:
positive_data_lemmatized = positive_data_lemmatized - wrong_tag_lemmatized

Al lematizar, es posible notar que hay una palabra menos que no tiene representación vectorial. En la siguiente celda, podrá evidenciarse qué palabra es la que hace la diferencia:

In [17]:
print(missing_words - missing_words_lemmatized)

{'spoilages'}


## Preprocesamiento de los datos

En las siguientes líneas, se iterará por sobre todas las palabras en los conjuntos de palabras reducidas a su lema (`positive_data_lemmatized` y `negative_data_lemmatized`), con el fin de encontrar su vector en la matriz de *embeddings* GloVe. Estos vectores se guardarán en las matrices `positive_matrix` y `negative_matrix`, que contendrán, respectivamente, las palabras de sentimiento positivo y negativo.

Es importante destacar el trato que reciben las palabras para las que no se dispone una representación vectorial. Tras [investigar](https://github.com/stanfordnlp/GloVe/search?utf8=%E2%9C%93&q=unk&type=), se descubrió que GloVe incluye un vector estándar para las palabras sin *word embedding*, el que en esta tarea puede obtenerse con `vec("unk")`. Así, a todas las palabras del *opinion lexicon* que no tienen *embedding* en GloVe, se les asigna el vector antes mencionado.

In [52]:
positive_matrix = np.empty((0, 300))
positive_data_lemmatized = list(positive_data_lemmatized)
for word in positive_data_lemmatized:
    if word in glove_words:
        positive_matrix = np.append(positive_matrix, [vec(word)], axis=0)
    else:
        positive_matrix = np.append(positive_matrix, [vec("unk")], axis=0)

In [53]:
negative_matrix = np.empty((0, 300))
negative_data_lemmatized = list(negative_data_lemmatized)
for word in negative_data_lemmatized:
    if word in glove_words:
        negative_matrix = np.append(negative_matrix, [vec(word)], axis=0)
    else:
        negative_matrix = np.append(negative_matrix, [vec("unk")], axis=0)

Se observa que las matrices recién formadas efectivamente contienen todas las representaciones vectoriales que corresponden:

In [54]:
print(len(positive_data_lemmatized), positive_matrix.shape)
print(len(negative_data_lemmatized), negative_matrix.shape)

1951 (1951, 300)
4492 (4492, 300)


Se definen las etiquetas para cada ejemplo en el *data set*.

In [21]:
positive_labels = [1] * positive_matrix.shape[0]
negative_labels = [0] * negative_matrix.shape[0]
all_labels = positive_labels + negative_labels

Finalmente, se concatenan las matrices `positive_matrix` y `negative_matrix`.

In [22]:
concatenated_matrix = np.concatenate([positive_matrix, negative_matrix], axis=0)
print(concatenated_matrix.shape)

(6443, 300)


## Clasificación de sentimiento de palabras

### Separación entre entrenamiento y *testing*

Con el objeto de hacer una correcta clasificación, se separan los datos en entrenamiento y *testing*. En particular, se utiliza el 80% de los datos para entrenar los modelos y 20% para probarlos.

In [55]:
all_words = positive_data_lemmatized + negative_data_lemmatized
X_train, X_test, y_train, y_test, words_train, words_test = train_test_split(concatenated_matrix, all_labels, all_words, test_size=0.2, random_state=0)

### Calibración de híper-parámetros

Para mejorar el desempeño de los modelos, se define la siguiente función para calibrar los híper-parámetros correspondientes. Esto se hace posible gracias a `GridSearchCV`, que recibe un modelo, los datos de entrenamiento, los híper-parámetros a testear y la cantidad de *folds* en que se separarán los datos (de acuerdo a la documentación, los datos se separán de forma que se conserven las proporciones originales de las clases). Con esta información, `GridSearchCV` probará todas las combinaciones de híper-parámetros y se realizará validación cruzada con ellas. Finalmente se retornará la combinación de híper-parámetros que haya obtenido mejor `cross_val_score`.

In [24]:
def estimator_grid_search(estimator, parameters):
    clf = GridSearchCV(estimator=estimator, param_grid=parameters, n_jobs=-1, cv=5)
    clf.fit(X_train, y_train)   
    print("Mejor accuracy: {}".format(clf.best_score_))
    return clf

### Medición del desempeño de los modelos

Para evaluar los modelos, es necesario cuantificar su desempeño. Por esta razón, se define la función `get_metrics` que dado un conjunto de entrenamiento con sus respectivas etiquetas, permite obtener un diccionario con las principales métricas para tareas de clasificación.

In [25]:
def get_metrics(clf, X_test, y_test):
    metrics = {}
    predictions = clf.predict(X_test)
    metrics["accuracy"] = accuracy_score(y_test, predictions)
    metrics["precision"] = precision_score(y_test, predictions)
    metrics["recall"] = recall_score(y_test, predictions)
    metrics["f1-score"] = f1_score(y_test, predictions)
    return metrics

### Guardar modelos

La función `save_model` permite guardar los clasificadores entrenados en esta tarea.

In [26]:
def save_model(clf, clf_name):
    joblib.dump(clf, "best_{}.pkl".format(clf_name))

### SVM


El primer modelo entrenado en esta tarea corresponde a *Support Vector Machines*. Para ajustarlo, primero se realiza la calibración de híper-parámetros. Una vez encontrada la mejor combinación de ellos, se entrena el clasificador con ellos y los datos de entrenamiento. Finalmente, se obtienen las métricas antes descritas sobre el set de *testing*. Como los datos de este conjunto no han sido vistos por el modelo, puede asumirse que las métricas son una buena muestra de la calidad del clasificador.

In [27]:
svm_params = {
    "C": [1, 10],
    "gamma": [0.001, 0.0001],
    "kernel": ["linear", "poly", "rbf", "sigmoid"]
}
tuning_svm = estimator_grid_search(SVC(), svm_params)
print("Mejor C: {}".format(tuning_svm.best_estimator_.C))
print("Mejor kernel: {}".format(tuning_svm.best_estimator_.kernel))
print("Mejor Gamma: {}".format(tuning_svm.best_estimator_.gamma))
final_svm = SVC(C=tuning_svm.best_estimator_.C, 
                kernel=tuning_svm.best_estimator_.kernel, 
                gamma=tuning_svm.best_estimator_.gamma, probability=True)
final_svm.fit(X_train, y_train)
svm_final_metrics = get_metrics(final_svm, X_test, y_test)
print(svm_final_metrics)
save_model(final_svm, "SVM")

Mejor accuracy: 0.9396585176561894
Mejor C: 10
Mejor kernel: rbf
Mejor Gamma: 0.001
{'accuracy': 0.9332816136539953, 'precision': 0.9141274238227147, 'recall': 0.8571428571428571, 'f1-score': 0.8847184986595175}


### KNN

Se realizan los mismos pasos que en el caso del SVM, pero ahora para entrenar un clasificador KNN.

In [28]:
knn_params = {"n_neighbors": np.arange(5) + 1, "algorithm": ["kd_tree", "ball_tree"]}
tuning_knn = estimator_grid_search(KNeighborsClassifier(), knn_params)
print("Mejor K: {}".format(tuning_knn.best_estimator_.n_neighbors)) 
print("Mejor algoritmo de búsqueda de vecinos: {}".format(tuning_knn.best_estimator_.algorithm))
final_knn = KNeighborsClassifier(n_neighbors=tuning_knn.best_estimator_.n_neighbors, 
                                 algorithm=tuning_knn.best_estimator_.algorithm)
final_knn.fit(X_train, y_train)
knn_final_metrics = get_metrics(final_knn, X_test, y_test)
print(knn_final_metrics)
save_model(final_knn, "KNN")

Mejor accuracy: 0.9086146682188592
Mejor K: 5
Mejor algoritmo de búsqueda de vecinos: kd_tree
{'accuracy': 0.921644685802948, 'precision': 0.9355828220858896, 'recall': 0.7922077922077922, 'f1-score': 0.8579465541490859}


### Random Forest

Finalmente, se evaluó el desempeño de un ensamble de árboles de decisión sobre esta tarea de clasificación.

In [29]:
rf_params = {
    "n_estimators": np.arange(10) + 1, 
    "criterion": ["gini", "entropy"], 
    "max_features": ["sqrt", "log2", None]}
tuning_rf = estimator_grid_search(RandomForestClassifier(), rf_params)
print("Mejor cantidad de árboles: {}".format(tuning_rf.best_estimator_.n_estimators))
print("Mejor criterio de división: {}".format(tuning_rf.best_estimator_.criterion))
print("Mejor cantidad máxima de features: {}".format(tuning_rf.best_estimator_.max_features))
final_rf = RandomForestClassifier(n_estimators=tuning_rf.best_estimator_.n_estimators,
                                 criterion=tuning_rf.best_estimator_.criterion,
                                 max_features=tuning_rf.best_estimator_.max_features)
final_rf.fit(X_train, y_train)
rf_final_metrics = get_metrics(final_rf, X_test, y_test)
print(rf_final_metrics)
save_model(final_rf, "RF")

Mejor accuracy: 0.8649592549476135
Mejor cantidad de árboles: 9
Mejor criterio de división: entropy
Mejor cantidad máxima de features: None
{'accuracy': 0.8580294802172227, 'precision': 0.821656050955414, 'recall': 0.6701298701298701, 'f1-score': 0.7381974248927038}


En el enunciado de la tarea, se pide que los modelos tengan salida continua. Es por eso que se define la función `model_predict`, que retorna la probabilidad de pertenencia a cada una de las clases (positivo/negativo).

In [30]:
def model_predict(clf, test_set):
    return clf.predict_proba(test_set)

Para comparar las salidas continuas con las verdaderas etiquetas de los datos, se crea el siguiente `DataFrame`:

In [72]:
svm_df = pd.DataFrame(model_predict(final_svm,  X_test), columns=["negative_probability", "positive_probability"])
knn_df = pd.DataFrame(model_predict(final_knn, X_test), columns=["negative_probability", "positive_probability"])
rf_df = pd.DataFrame(model_predict(final_rf, X_test), columns=["negative_probability", "positive_probability"])
svm_df["word"] = words_test
knn_df["word"] = words_test
rf_df["word"] = words_test
svm_df["true_label"] = y_test
knn_df["true_label"] = y_test
rf_df["true_label"] = y_test

svm_df = svm_df[["word", "negative_probability", "positive_probability", "true_label"]]
knn_df = knn_df[["word", "negative_probability", "positive_probability", "true_label"]]
rf_df = rf_df[["word", "negative_probability", "positive_probability", "true_label"]]

In [75]:
random_words = svm_df.sample(5)["word"].tolist()

display(svm_df[svm_df["word"].isin(random_words)])
display(knn_df[knn_df["word"].isin(random_words)])
display(rf_df[rf_df["word"].isin(random_words)])

,word,negative_probability,positive_probability,true_label
188,autocratic,0.996527,0.003473,0
416,sadness,0.947665,0.052335,0
455,fervor,0.024645,0.975355,1
924,refuse,0.999182,0.000818,0
985,sleazy,0.998606,0.001394,0


,word,negative_probability,positive_probability,true_label
188,autocratic,1.0,0.0,0
416,sadness,1.0,0.0,0
455,fervor,0.0,1.0,1
924,refuse,1.0,0.0,0
985,sleazy,1.0,0.0,0


,word,negative_probability,positive_probability,true_label
188,autocratic,0.888889,0.111111,0
416,sadness,1.000000,0.000000,0
455,fervor,0.222222,0.777778,1
924,refuse,0.777778,0.222222,0
985,sleazy,1.000000,0.000000,0


## Predicción de oraciones

En esta sección se detalla lo necesario para extender el poder de los modelos a la clasificación de oraciones de largo arbitrario.
### Preprocesamiento de las oraciones

Con el fin de eliminar las partes de las oraciones que no aportan demasiado a su significado, se realiza lo siguiente dentro de la función `tokenize_lemmatize`:
- Se convierte la oración recibida a minúsculas.
- Se eliminan los caracteres que no corresponden a letras (signos de puntuación y números).
- Se separa la oración en *tokens*.
- Se remueven las *stopwords*, palabras comunes que no contribuyen al significado de una oración.
- Lematización, tal como se explicó en una sección anterior.

In [33]:
def tokenize_lemmatize(sentence):
    sentence = sentence.lower()
    words_sentence = ''.join([c for c in sentence if c not in NON_WORDS])
    tokenized_sentence = word_tokenize(words_sentence)
    removed_stopwords = [word for word in tokenized_sentence if word not in ENGLISH_STOPWORDS]
    lemmatized_sentence = [nltk.stem.WordNetLemmatizer().lemmatize(word) for word in removed_stopwords]
    return tokenized_sentence

La obtención de la representación vectorial es similar a la utilizada para las palabras. En el caso de las oraciones, se obtiene el vector de cada palabra en la oración y se guarda en `sentence_matrix`. De no contar con el *embedding*, se utiliza el vector de "unk". Posteriormente, se promedian todas las coordenadas de los vectores en `sentence_matrix` y se guarda en `mean_sentence_vector`. Este será el vector que represente a una oración.

In [34]:
def vectorize_sentence(sentence):
    sentence_matrix = np.empty((0, 300))
    for word in sentence:
        if word in glove_words:
            sentence_matrix = np.append(sentence_matrix, [vec(word)], axis=0)
        else:
            sentence_matrix = np.append(sentence_matrix, [vec("unk")], axis=0)
    mean_sentence_vector = sentence_matrix.mean(0)
    if np.isnan(np.sum(mean_sentence_vector)):
        mean_sentence_vector = vec("unk")
    return mean_sentence_vector

La siguiente función ejecuta las dos funciones anteriores y retorna la representación vectorial de la oración entregada.

In [35]:
def prepare_sentence(sentence):
    tokenized = tokenize_lemmatize(sentence)
    vectorized = vectorize_sentence(tokenized)
    return vectorized

### Evaluación de los modelos sobre frases reales

Para evaluar el desempeño del modelo, se utiliza el *data set* [Sentiment Labelled Sentences](https://archive.ics.uci.edu/ml/datasets/Sentiment+Labelled+Sentences). Las siguientes líneas cargan y pre-procesan estos nuevos datos.

In [36]:
amazon = pd.read_csv("amazon_cells_labelled.txt", sep="\t", names=["sentence", "polarity"])
imdb = pd.read_csv("imdb_labelled.txt", sep="\t", names=["sentence", "polarity"])
yelp = pd.read_csv("yelp_labelled.txt", sep="\t", names=["sentence", "polarity"])

In [38]:
all_sentences = pd.concat([amazon, imdb, yelp])
all_sentences.head()

,sentence,polarity
0,So there is no way for me to plug it in here i...,0
1,"Good case, Excellent value.",1
2,Great for the jawbone.,1
3,Tied to charger for conversations lasting more...,0
4,The mic is great.,1


In [39]:
X_sentence_test = all_sentences["sentence"].tolist()
y_sentence_test = all_sentences["polarity"]

In [40]:
vectorized_sentences = list(map(prepare_sentence, X_sentence_test))

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:8: RuntimeWarning: Mean of empty slice.
  
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/numpy/core/_methods.py:73: RuntimeWarning: invalid value encountered in true_divide
  ret, rcount, out=ret, casting='unsafe', subok=False)


Lo anterior permite obtener las siguientes métricas: 

In [41]:
print(get_metrics(final_svm, vectorized_sentences, y_sentence_test))
print(get_metrics(final_knn, vectorized_sentences, y_sentence_test))
print(get_metrics(final_rf, vectorized_sentences, y_sentence_test))

{'accuracy': 0.5844250363901019, 'precision': 0.5490353697749196, 'recall': 0.9855699855699855, 'f1-score': 0.70521424883841}
{'accuracy': 0.5669577874818049, 'precision': 0.5411764705882353, 'recall': 0.9292929292929293, 'f1-score': 0.6840148698884758}
{'accuracy': 0.6670305676855895, 'precision': 0.6465463596764157, 'recall': 0.7496392496392497, 'f1-score': 0.6942866688940862}


## Reentrenamiento de los modelos

Se observa que el desempeño no es tan bueno como el que se tiene para la predicción de sentimiento de palabras. Por esta razón, se reentrenan los modelos anteriores con los datos para palabras y el 80% de los nuevos datos.

In [42]:
new_concatenated_matrix = np.concatenate([positive_matrix, negative_matrix, vectorized_sentences], axis=0)
new_all_labels = positive_labels + negative_labels + y_sentence_test.tolist()

In [61]:
words_and_sentences = all_words + X_sentence_test
new_X_train, new_X_test, new_y_train, new_y_test, new_words_train, new_words_test  = train_test_split(new_concatenated_matrix, new_all_labels, words_and_sentences, test_size=0.2, random_state=0)

In [44]:
new_svm = SVC(C=tuning_svm.best_estimator_.C, 
                kernel=tuning_svm.best_estimator_.kernel, 
                gamma=tuning_svm.best_estimator_.gamma, probability=True)
new_knn = KNeighborsClassifier(n_neighbors=tuning_knn.best_estimator_.n_neighbors, 
                                 algorithm=tuning_knn.best_estimator_.algorithm)
new_rf = RandomForestClassifier(n_estimators=tuning_rf.best_estimator_.n_estimators,
                                 criterion=tuning_rf.best_estimator_.criterion,
                                 max_features=tuning_rf.best_estimator_.max_features)

In [45]:
new_svm.fit(new_X_train, new_y_train)
new_knn.fit(new_X_train, new_y_train)
new_rf.fit(new_X_train, new_y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=9, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Así, se obtienen las siguientes métricas.

In [47]:
new_svm_metrics = get_metrics(new_svm, new_X_test, new_y_test)
new_knn_metrics = get_metrics(new_knn, new_X_test, new_y_test)
new_rf_metrics = get_metrics(new_rf, new_X_test, new_y_test)
print(new_svm_metrics)
print(new_knn_metrics)
print(new_rf_metrics)

{'accuracy': 0.912452419793366, 'precision': 0.9001610305958132, 'recall': 0.8495440729483282, 'f1-score': 0.874120406567631}
{'accuracy': 0.8504622077215879, 'precision': 0.8144499178981938, 'recall': 0.7537993920972644, 'f1-score': 0.7829518547750591}
{'accuracy': 0.8259923871669386, 'precision': 0.7934027777777778, 'recall': 0.6945288753799392, 'f1-score': 0.740680713128039}


Los siguientes `DataFrames` permiten visualizar las salidas continuas y la verdadera etiqueta de los datos de *testing*.

In [68]:
new_svm_df = pd.DataFrame(model_predict(new_svm,  new_X_test), columns=["negative_probability", "positive_probability"])
new_knn_df = pd.DataFrame(model_predict(new_knn, new_X_test), columns=["negative_probability", "positive_probability"])
new_rf_df = pd.DataFrame(model_predict(new_rf, new_X_test), columns=["negative_probability", "positive_probability"])
new_svm_df["word_sentence"] = new_words_test
new_knn_df["word_sentence"] = new_words_test
new_rf_df["word_sentence"] = new_words_test
new_svm_df["true_label"] = new_y_test
new_knn_df["true_label"] = new_y_test
new_rf_df["true_label"] = new_y_test
new_svm_df = new_svm_df[["word_sentence", "negative_probability", "positive_probability", "true_label"]]
new_knn_df = new_knn_df[["word_sentence", "negative_probability", "positive_probability", "true_label"]]
new_rf_df = new_rf_df[["word_sentence", "negative_probability", "positive_probability", "true_label"]]

In [71]:
random_words_sentences = new_svm_df.sample(5)["word_sentence"].tolist()

display(new_svm_df[new_svm_df["word_sentence"].isin(random_words_sentences)])
display(new_knn_df[new_knn_df["word_sentence"].isin(random_words_sentences)])
display(new_rf_df[new_rf_df["word_sentence"].isin(random_words_sentences)])

,word_sentence,negative_probability,positive_probability,true_label
168,morbidly,0.971653,0.028347,0
222,"I don't each much pasta, but I love the homema...",0.277592,0.722408,1
457,undefined,0.999932,0.000068,0
716,ingrate,0.990158,0.009842,0
1627,sputter,0.993942,0.006058,0


,word_sentence,negative_probability,positive_probability,true_label
168,morbidly,1.0,0.0,0
222,"I don't each much pasta, but I love the homema...",0.6,0.4,1
457,undefined,1.0,0.0,0
716,ingrate,1.0,0.0,0
1627,sputter,1.0,0.0,0


,word_sentence,negative_probability,positive_probability,true_label
168,morbidly,0.888889,0.111111,0
222,"I don't each much pasta, but I love the homema...",0.111111,0.888889,1
457,undefined,0.777778,0.222222,0
716,ingrate,1.000000,0.000000,0
1627,sputter,0.666667,0.333333,0


Finalmente, se serializan los modelos para referencias futuras.

In [50]:
save_model(new_svm, "sentences_SVM")
save_model(new_knn, "sentences_KNN")
save_model(new_rf, "sentences_RF")